In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from lsst.afw import cameraGeom
from lsst.obs.lsst.cameraTransforms import LsstCameraTransforms
from lsst.obs.lsst import LsstCam
camera = LsstCam.getCamera()

In [ ]:
def focal_to_pixel(fpx, fpy, det):
    """
    Parameters
    ----------
    fpx, fpy : array
        Focal plane position in millimeters in DVCS
        See https://lse-349.lsst.io/
    det : lsst.afw.cameraGeom.Detector
        Detector of interest.

    Returns
    -------
    x, y : array
        Pixel coordinates.
    """
    tx = det.getTransform(cameraGeom.FOCAL_PLANE, cameraGeom.PIXELS)
    x, y = tx.getMapping().applyForward(np.vstack((fpx, fpy)))
    return x.ravel(), y.ravel()

def pixel_to_focal(x, y, det):
    """
    Parameters
    ----------
    x, y : array
        Pixel coordinates.
    det : lsst.afw.cameraGeom.Detector
        Detector of interest.

    Returns
    -------
    fpx, fpy : array
        Focal plane position in millimeters in DVCS
        See https://lse-349.lsst.io/
    """
    tx = det.getTransform(cameraGeom.PIXELS, cameraGeom.FOCAL_PLANE)
    fpx, fpy = tx.getMapping().applyForward(np.vstack((x, y)))
    return fpx.ravel(), fpy.ravel()

In [ ]:
lct = {}
for detector in camera:
    if detector.getType()== cameraGeom.DetectorType.GUIDER :
        detName = detector.getName()
        lct[detector.getId()] = LsstCameraTransforms(camera,detName)  

In [ ]:
# given pixels ampX,ampY in Amp Pixel numbering , convert them to ccdX,ccdY in CCD pixel numbering

myDetName = 'R00_SG0'
col = 56
row = 800
roisize = 400

guiders = {}

for detector in camera:
    if detector.getType()== cameraGeom.DetectorType.GUIDER :
        detName = detector.getName()
        idet = detector.getId()
        bbox = detector.getBBox()
        nx,ny = bbox.getDimensions()
        guiders[idet] = np.zeros((ny,nx))
        if detName == myDetName:
            break

for amp in detector.getAmplifiers():
    ampName = amp.getName()

    llX = col
    llY = row
    urX = col + roisize
    urY = row + roisize

    llCCDX,llCCDY = lct[idet].ampPixelToCcdPixel(llX,llY,ampName)  # get CCD x,y these are floats
    llfpX,llfpY = lct[idet].ampPixelToFocalMm(llX,llY,ampName)     # get focal plane x,y in mm

    urCCDX,urCCDY = lct[idet].ampPixelToCcdPixel(urX,urY,ampName)
    urfpX,urfpY = lct[idet].ampPixelToFocalMm(urX,urY,ampName)

    # flip ROI as needed
    roiarr = np.ones((roisize,roisize))
    roiarr[0:100,0:100] =  2.0  # mark the corner closest to the readout
    
    if urCCDX < llCCDX:
        roiarr = np.fliplr(roiarr)
    if urCCDY < llCCDY:
        roiarr = np.flipud(roiarr)

    # this should place the ROI correctly inside the CCD
    guiders[idet][int(min(llCCDY,urCCDY)):int(max(llCCDY,urCCDY)),
        int(min(llCCDX,urCCDX)):int(max(llCCDX,urCCDX))] = roiarr

In [ ]:
print(detector.getName())
f,ax = plt.subplots(1,1)
_ = ax.imshow(guiders[189],origin='lower')
ax.axhline(2000, color='white')
for i in range(7):
    ax.axvline(509 * (i+1), color='white')
for amp in detector.getAmplifiers():
    ampName = amp.getName()
    labelX = 450
    labelY = 1500
    labelCCDX,labelCCDY = lct[idet].ampPixelToCcdPixel(labelX, labelY, ampName)  # get CCD x,y these are floats
    ax.text(labelCCDX, labelCCDY, ampName, color='white')


ampX=162.6  
ampY=950.8
ampName = 'C11'
X, Y = lct[idet].ampPixelToCcdPixel(ampX, ampY, ampName)
ax.text(X, Y, 'X', color='red')

ampX=194.4  
ampY=1416.2
ampName = 'C02'
X, Y = lct[idet].ampPixelToCcdPixel(ampX, ampY, ampName)
ax.text(X, Y, 'X', color='magenta')